<a href="https://colab.research.google.com/github/vardhanreddy2003/GPT-2Training/blob/main/LayerNormandGeLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
import torch.nn as nn

In [3]:
class LayerNorm(nn.Module):
  def __init__(self,emb_dim):
    super().__init__()
    self.eps=1e-5
    self.scale=nn.Parameter(torch.ones(emb_dim))
    self.shift=nn.Parameter(torch.zeros(emb_dim))

  def forward(self,x):
    mean=x.mean(dim=-1,keepdim=True)
    variance=x.vairance(dim=-1,keepdim=True)
    norm_x=(x-mean)/torch.sqrt(variance+self.eps)
    return self.scale*norm_x+self.shift


In [5]:
class GELU(nn.Module):
  def __init__(self):
     super().__init__()

  def forward(self,x):

    return 0.5*x*(1+torch.tanh(torch.sqrt(2.0/torch/pi))*(x+0.044715*torch.pow(x,3)))


